In [1]:
import pandas as pd
import numpy as np

In [2]:
from flask import Flask, send_file
from config import *

In [3]:
import datetime as dt
from datetime import time
from datetime import timedelta  

In [4]:
import sys

In [5]:
sys.path.append('../')

In [6]:
import argparse
import time
from sys import platform

from models import *
from utils.datasets import *
from utils.utils import *

In [7]:
cfg = '../cfg/yolov3.cfg'
weights = '../weights/yolov3.weights'
images = '../data/samples'
output='../output'
img_size=416
conf_thres=0.3
nms_thres=0.45
save_txt=False
save_images=True
webcam=False

In [8]:
parse_data_cfg('../cfg/coco.data')

{'gpus': '0,1,2,3',
 'num_workers': '10',
 'classes': '80',
 'train': '../coco/trainvalno5k.txt',
 'valid': '../coco/5k.txt',
 'names': 'data/coco.names',
 'backup': 'backup/',
 'eval': 'coco'}

In [9]:
device = torch_utils.select_device()
model = Darknet(cfg, img_size)
_ = load_darknet_weights(model, weights)
model.to(device).eval()

classes = ['person', 'non person']
colors = [[random.randint(0, 255) for _ in range(3)] for _ in range(len(classes))]

Found 2 GPUs
Using cuda _CudaDeviceProperties(name='GeForce GTX 1080 Ti', major=6, minor=1, total_memory=11176MB, multi_processor_count=28)



In [10]:
dataloader = LoadImages(images, img_size=img_size)

In [11]:
PERSON_FOUND = 0

def predict_image():
    global PERSON_FOUND
    
    for i, (path, img, im0) in enumerate(dataloader):
        save_path = '../output/pic.jpg'
        
        # Get detections
        img = torch.from_numpy(img).unsqueeze(0).to(device)
        pred = model(img)
        pred = pred[pred[:, :, 4] > conf_thres]  # remove boxes < threshold

        if len(pred) > 0:
            # Run NMS on predictions
            detections = non_max_suppression(pred.unsqueeze(0), conf_thres, nms_thres)[0]

            # Rescale boxes from 416 to true image size
            scale_coords(img_size, detections[:, :4], im0.shape).round()

            # Print results to screen
            # class 0 is person
            unique_classes = detections[:, -1].cpu().unique()
            detections = detections[detections[:, -1].cpu() == 0]

            person_count = (detections[:, -1].cpu() == 0).sum().numpy()
#             print('Found ', person_count, ' persons')


            # Draw bounding boxes and labels of detections
            for *xyxy, conf, cls_conf, cls in detections:
                # Add bbox to the image
                label = '%s %.2f' % (classes[int(cls)], conf)
                plot_one_box(xyxy, im0, label=label, color=colors[int(cls)])
        else:
            person_count = 0
            
        print('Found ', person_count, ' persons')
        
        PERSON_FOUND = person_count

        if save_images:  # Save generated image with detections
            cv2.imwrite(save_path, im0)
        break

In [12]:
predict_image()

Found  0  persons


In [13]:
ts = pd.read_hdf('../prediction.h5')

In [59]:
def make_prediction(date):
    
    time_zone = timedelta(hours=3)
    date += time_zone
    
    weekday = 1 # date.weekday()
    hour = 12  # date.hour
    minute = 20 # date.minute
    
    weekday = date.weekday()
    hour = date.hour 
    minute = date.minute
    
    print(weekday, hour, minute)
    
    if weekday not in list(range(0, 6)):
        result += "Today canteen doesn't work"
        return result
    weekday += 1

    today_ts = ts.loc[ts.weekday == weekday]
    time_start = dt.time(hour, minute)
    
    result = f"⏱Current time is: {time_start.strftime('%H:%M')}\n👀Person found: {PERSON_FOUND}\n\n"
    
    flag = False
    weekday += 1

    intervals = [15 for i in range(1, 5)]

    i = 0
    for interval in intervals:
        i += 1
        delta = timedelta(minutes=interval)
        time_end = (dt.datetime.combine(dt.date(1,1,1), time_start) + delta).time()
        split = today_ts.between_time(time_start, time_end)
        
        if split.shape[0] == 0 and hour > 12:
            result += "Canteen close at 16:00"
            break
            
        elif split.shape[0] == 0 and hour <= 12:
            if not flag:
                result += "Dinner starts at 11:00\n"
                flag = True
            
        
        min_people = int(np.min(split.y.values))
        min_time = split.y.idxmin().time().strftime('%H:%M')
        mean_load = int(np.mean(split.y.values))
        from_str = time_start.strftime('%H:%M')
        to_str = time_end.strftime('%H:%M')

        romb = '🔹' if i % 2 == 1 else '🔸'
        result += f'{romb}From {from_str} to {to_str} a minumum expected amount of people is {min_people}. This load is expected at {min_time}, while in average there will be {mean_load} persons.\n\n'
#         result += f'From {from_str} to {to_str}:\
#               \n\tmin people = {min_people},\
#               \n\tbest time = {min_time},\
#               \n\tmean load = {mean_load}' + '\n\n'

        time_start = time_end
    return result

In [60]:
print(make_prediction(dt.datetime.now()))

0 12 11
⏱Current time is: 12:11
👀Person found: 14

🔹From 12:11 to 12:26 a minumum expected amount of people is 2. This load is expected at 12:24, while in average there will be 7 persons.

🔸From 12:26 to 12:41 a minumum expected amount of people is 4. This load is expected at 12:26, while in average there will be 8 persons.

🔹From 12:41 to 12:56 a minumum expected amount of people is 6. This load is expected at 12:47, while in average there will be 9 persons.

🔸From 12:56 to 13:11 a minumum expected amount of people is 8. This load is expected at 13:04, while in average there will be 11 persons.




In [61]:
app = Flask(__name__)

@app.route("/")
def hello():
    return "Hello World!"

@app.route("/picture")
def picture():
    return send_file("../data/samples/input.jpg", mimetype='image/jpeg')

@app.route("/labeled-picture")
def labaled_picture():
    predict_image()
    return send_file("../output/pic.jpg", mimetype='image/jpeg')

@app.route("/predict")
def predict():
    print(dt.datetime.now())
    return make_prediction(dt.datetime.now())

In [ ]:
app.run()

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [22/Apr/2019 09:11:28] "GET /labeled-picture HTTP/1.1" 200 -
127.0.0.1 - - [22/Apr/2019 09:11:28] "GET /predict HTTP/1.1" 200 -


Found  19  persons
2019-04-22 09:11:28.363570
0 12 11


127.0.0.1 - - [22/Apr/2019 09:11:51] "GET /picture HTTP/1.1" 200 -
127.0.0.1 - - [22/Apr/2019 09:11:51] "GET /predict HTTP/1.1" 200 -


2019-04-22 09:11:51.743569
0 12 11


127.0.0.1 - - [22/Apr/2019 09:17:14] "GET /labeled-picture HTTP/1.1" 200 -
127.0.0.1 - - [22/Apr/2019 09:17:14] "GET /predict HTTP/1.1" 200 -


...

2019-04-22 09:57:17.758796
0 12 57


127.0.0.1 - - [22/Apr/2019 09:57:40] "GET /picture HTTP/1.1" 200 -
127.0.0.1 - - [22/Apr/2019 09:57:40] "GET /predict HTTP/1.1" 200 -


2019-04-22 09:57:40.342269
0 12 57


127.0.0.1 - - [22/Apr/2019 09:57:46] "GET /labeled-picture HTTP/1.1" 200 -
127.0.0.1 - - [22/Apr/2019 09:57:46] "GET /predict HTTP/1.1" 200 -


Found  4  persons
2019-04-22 09:57:46.372363
0 12 57
